### Graded Lab 4

Hello ! Welcome to Graded Lab of Module 4.

In the last assignment we had worked on correlation , hypothesis which business wanted us to test.
Its high time when we look at the missing values & outliers stuff.
**Here we will work on missing values & outliers.**

In case if you are not able to recollect the problem description and data description then mentioning it below.

Lets look at the problem statement,

*Client: ABC Retail, Incorporated, rest-of-the-world division* 

***Project name: Online retail sales analysis*** 

An online retailer, ABC, Inc., operates in nearly 100 countries worldwide, selling furniture, office supplies and technology products to customers in three segments: consumer, corporate and home office. ABC, Inc. is a US-based company, and it has two major divisions: US and rest of the world. We are working with the rest of the world division of the company. 

They have provided us with online sales transaction data from 2011 to 2014.

We are given 3 datasets:-

1. Data on each sale; 51290 records; all data in US dollars
It contains fields like
**order_id** (identifier) ,order_date ,ship_date ,ship_mode ,**customer_id**(identifier) ,product_id ,category ,sub_category ,product_name ,sales ,quantity ,discount ,profit ,shipping_cost ,order_priority ,**vendor_code** (identifier) 


2. Data on the customers; 1590 records 
It contains fields like
**customer_id** (identifier) ,customer_name ,city ,state ,country ,postal_code ,segment ,market ,region 

3. Data on vendors who supply the retailer; 65 records 
It contains fields like
vendor ,**vendor_code** (identifier) 

We need to analyze the data and need to provide answer to different questions asked by company officials.

In [1]:
# importing libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import norm
from itertools import combinations
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

### Reading sales data
sales = pd.read_csv('sales_data_M4.csv')

### Reading customer data
cust = pd.read_csv(r'customers.csv',encoding='iso-8859-1')

### Reading vendor data
vend = pd.read_csv(r'vendors.csv')

sales.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,product_id,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,vendor_code
0,1,MX-2014-143658,02/10/2014,06/10/2014,Standard Class,SC-20575,OFF-LA-10002782,Office Supplies,Labels,"Hon File Folder Labels, Adjustable",13.08,3,0.0,4.56,1.03,Medium,VE_001
1,2,MX-2012-155047,15/10/2012,20/10/2012,Standard Class,KW-16570,FUR-FU-10004015,Furniture,Furnishings,"Tenex Clock, Durable",252.16,8,0.0,90.72,13.45,Medium,VE_002
2,3,MX-2012-155047,15/10/2012,20/10/2012,Standard Class,KW-16570,FUR-BO-10002352,Furniture,Bookcases,"Ikea 3-Shelf Cabinet, Mobile",193.28,2,0.0,54.08,9.63,Medium,VE_003
3,4,MX-2012-155047,15/10/2012,20/10/2012,Standard Class,KW-16570,OFF-BI-10004428,Office Supplies,Binders,"Cardinal Binder, Clear",35.44,4,0.0,4.96,1.37,Medium,VE_004
4,5,MX-2012-155047,15/10/2012,20/10/2012,Standard Class,KW-16570,OFF-AR-10004594,Office Supplies,Art,"Sanford Canvas, Water Color",71.60,2,0.0,11.44,3.79,Medium,VE_005


In [2]:
pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format','{:.2f}'.format)

In [3]:
cust.head()

,customer_id,customer_name,city,state,country,postal_code,segment,market,region
0,SC-20575,Sonia Cooley,Mexico City,Distrito Federal,Mexico,NaN,Consumer,LATAM,LATAM-North
1,KW-16570,Kelly Williams,Dos Quebradas,Risaralda,Colombia,NaN,Consumer,LATAM,LATAM-South
2,DP-13000,Darren Powers,São Paulo,São Paulo,Brazil,NaN,Consumer,LATAM,LATAM-South
3,TB-21250,Tim Brockman,Managua,Managua,Nicaragua,NaN,Consumer,LATAM,LATAM-Central
4,PK-18910,Paul Knutson,Managua,Managua,Nicaragua,NaN,Home Office,LATAM,LATAM-Central


In [4]:
vend.head()

,vendor,vendor_code
0,Accenco,VE_001
1,Ace Corporation,VE_002
2,Nero,VE_003
3,Rushcorp,VE_004
4,Low Tide Corp,VE_005


In order to solve the next questions , we need to combine all the 3 datasets into a single dataframe such that every details of sales dataframe are intact. So here we have written a data processing function.
There are 2 tasks which are to be performed.
1. Merge/ Join all the 3 datasets into a single dataframe such that every details of sales dataframe are intact. (Understand which should be the joining key , type of join , refer .merge() function of pandas)
2. Convert 'order_date' into a datetime column.
**Return output as a dataframe**

In [5]:
#### data_merging & order_date processing , data1 will be sales , data2 will be customer dataset & data3 will be vendor dataset.

def data_process(data1,data2,data3):
    df = data1.merge(right=data3, how="left", on="vendor_code")
    df2 = df.merge(right=data2, how="left", on="customer_id")
    df2["order_date"] = pd.to_datetime(df2['order_date'], format='%d/%m/%Y')
    return df2

In [6]:
sales= data_process(data1=sales.copy(),data2=cust.copy(),data3=vend.copy())

In [7]:
assert sales['order_date'].dtypes=='<M8[ns]' ,'Make sure if you have converted order_date into a datetime format correctly or not.'
assert sales.shape== (51290,26) ,'Checking size and shape of dataframe after merging is a very important check.'

In [8]:
sales.columns

Index(['row_id', 'order_id', 'order_date', 'ship_date', 'ship_mode', 'customer_id', 'product_id', 'category', 'sub_category', 'product_name', 'sales', 'quantity', 'discount', 'profit', 'shipping_cost', 'order_priority', 'vendor_code', 'vendor', 'customer_name', 'city', 'state', 'country', 'postal_code', 'segment', 'market', 'region'], dtype='object')

Note that in this dataset we are purposefully introducing some of the missing values.

### Q.1 Return column names & missing values percentage for the columns which have % misisng value >50. Output should be a dictionary. For eg:- { column_name : %}, make sure to round off percentages to 2 decimals.

In [9]:
totalna = sales.isnull().sum().sum()
totalna

64978

In [10]:
missingdf = pd.DataFrame(sales.isnull().sum(), columns=["no"]).reset_index()
missingdf

,index,no
0,row_id,0
1,order_id,0
2,order_date,0
3,ship_date,0
4,ship_mode,1570
5,customer_id,0
6,product_id,0
7,category,3571
8,sub_category,0
9,product_name,1297


In [11]:
missingdf["%"] = round((missingdf["no"]/totalna * 100),2)

In [12]:
missingdf

,index,no,%
0,row_id,0,0.00
1,order_id,0,0.00
2,order_date,0,0.00
3,ship_date,0,0.00
4,ship_mode,1570,2.42
5,customer_id,0,0.00
6,product_id,0,0.00
7,category,3571,5.50
8,sub_category,0,0.00
9,product_name,1297,2.00


In [13]:
missingdf.loc["postal_code"]["%"]

KeyError: 'postal_code'

### ----------------------------------------------------------

In [14]:
def missing_value_col(data):
    # your code here
    # Calculate missing value percentages for each column
    missing_percentage = (data.isnull().sum() / len(data)) * 100
    
    # Filter columns with missing values > 50%
    missing_columns = missing_percentage[missing_percentage > 50]
    
    # Round off percentages to 2 decimals
    missing_columns = missing_columns.round(2)

    # Convert the result to a dictionary
    missing_columns_dict = missing_columns.to_dict()
    
    return missing_columns_dict

In [15]:
# autograder cells , please do not alter/ delete /edit this cell

In [16]:
missing_value_col(data=sales)

{'postal_code': 100.0}

#### **Senior data scientists after consulting with the business have decided that such columns needs to be dropped before going further.**

In [17]:
high_missing_col=list(missing_value_col(data=sales).keys()) #Enter the columns with high % of missing values
### We will drop columns with such high missing values
sales.drop(high_missing_col,axis=1,inplace=True)

In [18]:
sales.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,product_id,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,vendor_code,vendor,customer_name,city,state,country,segment,market,region
0,1,MX-2014-143658,2014-10-02,06/10/2014,Standard Class,SC-20575,OFF-LA-10002782,Office Supplies,Labels,"Hon File Folder Labels, Adjustable",13.08,3,0.00,4.56,1.03,Medium,VE_001,Accenco,Sonia Cooley,Mexico City,Distrito Federal,Mexico,Consumer,LATAM,LATAM-North
1,2,MX-2012-155047,2012-10-15,20/10/2012,Standard Class,KW-16570,FUR-FU-10004015,Furniture,Furnishings,"Tenex Clock, Durable",252.16,8,0.00,90.72,13.45,Medium,VE_002,Ace Corporation,Kelly Williams,Dos Quebradas,Risaralda,Colombia,Consumer,LATAM,LATAM-South
2,3,MX-2012-155047,2012-10-15,20/10/2012,Standard Class,KW-16570,FUR-BO-10002352,Furniture,Bookcases,"Ikea 3-Shelf Cabinet, Mobile",193.28,2,0.00,54.08,9.63,Medium,VE_003,Nero,Kelly Williams,Dos Quebradas,Risaralda,Colombia,Consumer,LATAM,LATAM-South
3,4,MX-2012-155047,2012-10-15,20/10/2012,Standard Class,KW-16570,OFF-BI-10004428,Office Supplies,Binders,"Cardinal Binder, Clear",35.44,4,0.00,4.96,1.37,Medium,VE_004,Rushcorp,Kelly Williams,Dos Quebradas,Risaralda,Colombia,Consumer,LATAM,LATAM-South
4,5,MX-2012-155047,2012-10-15,20/10/2012,Standard Class,KW-16570,OFF-AR-10004594,Office Supplies,Art,"Sanford Canvas, Water Color",71.60,2,0.00,11.44,3.79,Medium,VE_005,Low Tide Corp,Kelly Williams,Dos Quebradas,Risaralda,Colombia,Consumer,LATAM,LATAM-South


### Q.2 Fill the missing data in 'category' column with appropriate method.  
(Hint:- Remember variable 'category' & 'sub-category' are related & hierachy goes like this category -> sub-category)

Approach:-

1. Create a key-pair dictionary where key will be product category & pair values will be sub-categories.

2. Wherever category value is missing check for corresponding sub-category value. 

3. From sub-category value trace back its original product category value by looking in dictionary, get a product value.

4. Fill the blank product category value by the same.

In [19]:
### create a dictionary where keys will be categories & sub-categories as values. store in inside Category_dict.

def cat_dict(data,main_col,sub_col):    
    Category_dict = {}
    for index, row in data.iterrows():
        if not pd.isna(row[main_col]) and not pd.isna(row[sub_col]):
            Category_dict[row[main_col]] = Category_dict.get(row[main_col], []) + [row[sub_col]]
    return Category_dict

def category_subcategory_map_dataframe(df, Category_dict, main_col, sub_col):
    def category_subcategory_map(row):
        if pd.isna(row[main_col]):  # If the main column is null
            if not pd.isna(row[sub_col]):  # If the sub column is not null
                for k, v in Category_dict.items():
                    if row[sub_col] in Category_dict[k]:
                        return k  # k is the correct sector for the category value v
        else:
            return row[main_col]  # If the main column is not null, return its existing value

    df[main_col] = df.apply(category_subcategory_map, axis=1)
    return df[main_col]

In [20]:
Category_dict=cat_dict(data=sales,main_col='category',sub_col='sub_category')
sales['category']=category_subcategory_map_dataframe(df=sales, Category_dict=Category_dict, main_col='category', sub_col='sub_category')

In [21]:
sales.category.isnull().sum()

0

In [22]:
# autograder cells , please do not alter/ delete /edit this cell

### Q.3 ) Fill the missing values in sales column by mean. Return output series.

In [23]:
def fill_missing(data,col):
    mean_value = data[col].mean()
    data[col] = data[col].fillna(mean_value)
    return data[col]

In [24]:
sales['sales']=fill_missing(data=sales,col='sales')

In [25]:
sales.sales.isnull().sum()

0

In [26]:
# autograder cells , please do not alter/ delete /edit this cell

### Q.4 ) How many orders have sales falling in the following groups 
Groups :- [0,12),[12,37),[37,113),[113,186),[186,1200),[1200,max).

**Note :-[a,b) indicates that bin includes a but excudes b]**

**Return output series**

In [27]:
sales["sales"].describe()

count   51290.00
mean      246.43
std       451.85
min         0.44
25%        36.00
50%       111.12
75%       246.43
max     22638.48
Name: sales, dtype: float64

In [28]:
def manual_bin(data,col):
    # Define the bin edges
    bins = [0, 12, 37, 113, 186, 1200, float('inf')]
    
    # Use the cut function to categorize 'sales' into the specified bins
    data['bin'] = pd.cut(data[col], bins, right=False)
    
    # Use value_counts to count the number of orders in each bin
    bin_counts = data['bin'].value_counts().sort_index()

    # Reset the index to have a clean output series
    bin_counts = bin_counts.reset_index()

    # Rename the columns for clarity
    # bin_counts.columns = ['Sales Range', 'Number of Orders']
    bin_counts = pd.Series(data=bin_counts["bin"])
    
    return bin_counts

In [29]:
manual_bin(sales, "sales")

0     3708
1     9325
2    12761
3     5187
4    18550
5     1759
Name: bin, dtype: int64

In [30]:
assert type(manual_bin(data=sales,col='sales'))==pd.Series, "Please provide output in series format."

In [31]:
# autograder cells , please do not alter/ delete /edit this cell

### Q.5 )  Sometimes, it is difficult to comprehend the sales value as a numeric value. So, the business team wants to observe the sales as Low, Medium and High categories.  Write a code to discretize the sales data and  return a series containing Low, Medium and High with the respective number of records.

In [32]:
sales.quantity.value_counts()

2     12748
3      9682
1      8963
4      6385
5      4882
6      3020
7      2385
8      1361
9       987
10      276
14      186
12      176
11      156
13       83
Name: quantity, dtype: int64

In [33]:
def discrete_bin(data,col):
    # Define the bin edges and labels
    bins = [1, 5, 10, 14]
    labels = ['Low', 'Medium', 'High']
    
    # Use the cut function to categorize 'sales' into the specified bins and labels
    data['category'] = pd.cut(data[col], bins, labels=labels)
    
    # Use value_counts to count the number of records in each category
    category_counts = data['category'].value_counts().sort_index()
    
    # Reset the index to have a clean output series
    category_counts = category_counts.reset_index()
    
    category_counts = pd.Series(data=category_counts["category"])
    
    return category_counts

In [34]:
discrete_bin(sales,'quantity')

0    33697
1     8029
2      601
Name: category, dtype: int64

In [35]:
assert type(discrete_bin(data=sales,col='quantity'))==pd.Series, "Please provide output in series format."

In [36]:
# autograder cells , please do not alter/ delete /edit this cell

### Q.6. In order to perform further analysis, we intend to utilize the 'order_priority' variable, which has categorical values ('First Class'(0) > 'Second Class'(1) > 'Standard Class'(2) > 'Same Day'(3)). To facilitate this analysis, we need to convert 'order_priority' into a numerical feature. Please encode 'order_priority' into a numerical format.Return output series.

In [37]:
sales.ship_mode.value_counts()

Standard Class    29842
Second Class       9985
First Class        7272
Same Day           2621
Name: ship_mode, dtype: int64

In [38]:
## Select Appropriate priority

priority= ['Standard Class', 'Second Class', 'First Class', 'Same Day']

# Mapping dictionary to encode 'order_priority'
order_priority_mapping = {
    'Same Day': 3,
    'First Class': 0,
    'Second Class': 1,
    'Standard Class': 2  }

def transforming_ordered_var(data,col,priority,nan_val):
    data[col] = data[col].map(order_priority_mapping)
    data[col] = data[col].fillna(nan_val)
    return data[col]

In [39]:
transforming_ordered_var(sales,'ship_mode',priority,5)

0       2.00
1       2.00
2       2.00
3       2.00
4       2.00
        ... 
51285   0.00
51286   5.00
51287   0.00
51288   0.00
51289   2.00
Name: ship_mode, Length: 51290, dtype: float64

In [40]:
sales['shipping_mode_encoded']=transforming_ordered_var(data=sales,col='ship_mode',priority=priority,nan_val=5)

In [41]:
# autograder cells , please do not alter/ delete /edit this cell

### Q.8) Outliers can be defined in different ways. For each of the following definitions of outliers, what percent of the values in the  'sales' are considered outliers? Give the answer as a percent rounded upto two decimal places.

Note -
1. 'IQR_detection' : Less than (Q1 - 1.5*IQR) and greater than (Q3 + 1.5*IQR) are considered outliers

2. 'percentile_detection': Below the 3rd percentile and above the 97th percentile are considered outliers

3. 'mean_SD' : Less than (mean - 3*SD) and greater than (mean + 3*SD) are considered outliers 

In [42]:
sales["sales"]

0        13.08
1       252.16
2       193.28
3        35.44
4        71.60
         ...  
51285    11.07
51286    61.44
51287    80.52
51288   130.44
51289   246.43
Name: sales, Length: 51290, dtype: float64

In [43]:
def detect_outliers(data,col,method):
    if method=='IQR_detection':
        # Calculate the first quartile (Q1) and the third quartile (Q3)
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        
        # Calculate the interquartile range (IQR)
        IQR = Q3 - Q1
        
        # Determine the lower and upper bounds for outliers
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Count the number of outliers
        outliers = (data[col] < lower_bound) | (data[col] > upper_bound)
        
    elif method=='percentile_detection':
        
        # Calculate the 3rd and 97th percentiles
        percentile_3 = data[col].quantile(0.03)
        percentile_97 = data[col].quantile(0.97)
        
        # Determine the lower and upper bounds for outliers
        lower_bound = percentile_3
        upper_bound = percentile_97
        
        # Count the number of outliers
        outliers = (data[col] < lower_bound) | (data[col] > upper_bound)
        
    elif method=='mean_SD':
        # Calculate the mean and standard deviation (SD)
        mean = data[col].mean()
        std_dev = data[col].std()
        
        # Determine the lower and upper bounds for outliers
        lower_bound = mean - 3 * std_dev
        upper_bound = mean + 3 * std_dev
        
        # Count the number of outliers
        outliers = (data[col] < lower_bound) | (data[col] > upper_bound)
        
    outliers_perc = float((outliers.sum() / len(data[col])) * 100)
    
    outliers_perc = round(outliers_perc,2)
            
    return outliers_perc

# Assert the type of the output
# assert type(detect_outliers(data=data['sales'], col='sales', method='IQR_detection')) == np.float

In [44]:
detect_outliers(sales,'sales','IQR_detection')

10.06

In [45]:
assert type(detect_outliers(data=sales,col='sales',method='IQR_detection'))==np.float, 'Make sure that output is returned as float'

AttributeError: module 'numpy' has no attribute 'float'.
`np.float` was a deprecated alias for the builtin `float`. To avoid this error in existing code, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [46]:
# autograder cells , please do not alter/ delete /edit this cell

### Q.10) Detect upper end outliers in profit column by 'IQR_detection'. Return number of upper end outliers.

In [47]:
def detect_upperend_outliers(data,col):
    
    # Calculate the first quartile (Q1) and the third quartile (Q3)
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
        
    # Calculate the interquartile range (IQR)
    IQR = Q3 - Q1
        
    # Determine the lower and upper bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
        
    # Count the number of outliers
    upper_outliers = int((data[col] > upper_bound).sum())
        
    return upper_outliers

In [48]:
detect_upperend_outliers(sales,'profit')

6229

In [49]:
assert type(detect_upperend_outliers(data=sales,col='profit'))==int , 'No of outliers should be an integer , make sure that you are returning output in integer format.'

In [50]:
# autograder cells , please do not alter/ delete /edit this cell

### Q.11) Floor & Capping outliers at 97% at 3%. Return the processed column as output.

In [51]:
sales.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,product_id,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,vendor_code,vendor,customer_name,city,state,country,segment,market,region,bin,shipping_mode_encoded
0,1,MX-2014-143658,2014-10-02,06/10/2014,5.00,SC-20575,OFF-LA-10002782,Low,Labels,"Hon File Folder Labels, Adjustable",13.08,3,0.00,4.56,1.03,Medium,VE_001,Accenco,Sonia Cooley,Mexico City,Distrito Federal,Mexico,Consumer,LATAM,LATAM-North,"[12.0, 37.0)",5.00
1,2,MX-2012-155047,2012-10-15,20/10/2012,5.00,KW-16570,FUR-FU-10004015,Medium,Furnishings,"Tenex Clock, Durable",252.16,8,0.00,90.72,13.45,Medium,VE_002,Ace Corporation,Kelly Williams,Dos Quebradas,Risaralda,Colombia,Consumer,LATAM,LATAM-South,"[186.0, 1200.0)",5.00
2,3,MX-2012-155047,2012-10-15,20/10/2012,5.00,KW-16570,FUR-BO-10002352,Low,Bookcases,"Ikea 3-Shelf Cabinet, Mobile",193.28,2,0.00,54.08,9.63,Medium,VE_003,Nero,Kelly Williams,Dos Quebradas,Risaralda,Colombia,Consumer,LATAM,LATAM-South,"[186.0, 1200.0)",5.00
3,4,MX-2012-155047,2012-10-15,20/10/2012,5.00,KW-16570,OFF-BI-10004428,Low,Binders,"Cardinal Binder, Clear",35.44,4,0.00,4.96,1.37,Medium,VE_004,Rushcorp,Kelly Williams,Dos Quebradas,Risaralda,Colombia,Consumer,LATAM,LATAM-South,"[12.0, 37.0)",5.00
4,5,MX-2012-155047,2012-10-15,20/10/2012,5.00,KW-16570,OFF-AR-10004594,Low,Art,"Sanford Canvas, Water Color",71.60,2,0.00,11.44,3.79,Medium,VE_005,Low Tide Corp,Kelly Williams,Dos Quebradas,Risaralda,Colombia,Consumer,LATAM,LATAM-South,"[37.0, 113.0)",5.00


In [52]:
def cap_and_floor_column(column, cap_percentile, floor_percentile):
    # Calculate the 3rd and 97th percentiles
    percentile_3 = column.quantile(0.03)
    percentile_97 = column.quantile(0.97)
        
    # Determine the lower and upper bounds for outliers
    lower_bound = percentile_3
    upper_bound = percentile_97
        
    # Cap and floor the column based on the bounds
    capped_and_floored_column = column.clip(lower=lower_bound, upper=upper_bound)
    
    return capped_and_floored_column ## Capped ,Floored series

In [53]:
sales['profit_outlier_capped'] = cap_and_floor_column(column=sales['profit'], cap_percentile=97, floor_percentile=3)

In [54]:
sales['profit_outlier_capped']

0        4.56
1       90.72
2       54.08
3        4.96
4       11.44
         ... 
51285    3.42
51286   18.42
51287   20.88
51288   33.84
51289   12.96
Name: profit_outlier_capped, Length: 51290, dtype: float64

In [55]:
sales

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,product_id,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,vendor_code,vendor,customer_name,city,state,country,segment,market,region,bin,shipping_mode_encoded,profit_outlier_capped
0,1,MX-2014-143658,2014-10-02,06/10/2014,5.00,SC-20575,OFF-LA-10002782,Low,Labels,"Hon File Folder Labels, Adjustable",13.08,3,0.00,4.56,1.03,Medium,VE_001,Accenco,Sonia Cooley,Mexico City,Distrito Federal,Mexico,Consumer,LATAM,LATAM-North,"[12.0, 37.0)",5.00,4.56
1,2,MX-2012-155047,2012-10-15,20/10/2012,5.00,KW-16570,FUR-FU-10004015,Medium,Furnishings,"Tenex Clock, Durable",252.16,8,0.00,90.72,13.45,Medium,VE_002,Ace Corporation,Kelly Williams,Dos Quebradas,Risaralda,Colombia,Consumer,LATAM,LATAM-South,"[186.0, 1200.0)",5.00,90.72
2,3,MX-2012-155047,2012-10-15,20/10/2012,5.00,KW-16570,FUR-BO-10002352,Low,Bookcases,"Ikea 3-Shelf Cabinet, Mobile",193.28,2,0.00,54.08,9.63,Medium,VE_003,Nero,Kelly Williams,Dos Quebradas,Risaralda,Colombia,Consumer,LATAM,LATAM-South,"[186.0, 1200.0)",5.00,54.08
3,4,MX-2012-155047,2012-10-15,20/10/2012,5.00,KW-16570,OFF-BI-10004428,Low,Binders,"Cardinal Binder, Clear",35.44,4,0.00,4.96,1.37,Medium,VE_004,Rushcorp,Kelly Williams,Dos Quebradas,Risaralda,Colombia,Consumer,LATAM,LATAM-South,"[12.0, 37.0)",5.00,4.96
4,5,MX-2012-155047,2012-10-15,20/10/2012,5.00,KW-16570,OFF-AR-10004594,Low,Art,"Sanford Canvas, Water Color",71.60,2,0.00,11.44,3.79,Medium,VE_005,Low Tide Corp,Kelly Williams,Dos Quebradas,Risaralda,Colombia,Consumer,LATAM,LATAM-South,"[37.0, 113.0)",5.00,11.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51285,51286,HU-2012-7730,2012-09-05,07/09/2012,5.00,NF-8385,OFF-AVE-10004570,NaN,Binders,"Avery Binder Covers, Economy",11.07,1,0.00,3.42,1.98,High,VE_047,Arcanestar,Natalie Fritzler,Montréal,Quebec,Canada,Consumer,Canada,Canada,"[0.0, 12.0)",5.00,3.42
51286,51287,HU-2012-7730,2012-09-05,07/09/2012,5.00,NF-8385,TEC-LOG-10004419,Low,Accessories,"Logitech Flash Drive, USB",61.44,2,0.00,18.42,13.02,High,VE_032,Padlock Microsystems,Natalie Fritzler,Montréal,Quebec,Canada,Consumer,Canada,Canada,"[37.0, 113.0)",5.00,18.42
51287,51288,HU-2012-7730,2012-09-05,07/09/2012,5.00,NF-8385,OFF-BOS-10002705,Low,Art,"Boston Highlighters, Fluorescent",80.52,4,0.00,20.88,8.78,High,VE_048,Stormking,Natalie Fritzler,Montréal,Quebec,Canada,Consumer,Canada,Canada,"[37.0, 113.0)",5.00,20.88
51288,51289,HU-2012-7730,2012-09-05,07/09/2012,5.00,NF-8385,OFF-ENE-10004132,Low,Paper,"Enermax Computer Printout Paper, 8.5 x 11",130.44,4,0.00,33.84,18.97,High,VE_047,Arcanestar,Natalie Fritzler,Montréal,Quebec,Canada,Consumer,Canada,Canada,"[113.0, 186.0)",5.00,33.84


In [56]:
assert round(np.quantile(sales['profit_outlier_capped'],0.97),4)==301.9693, 'Make sure that you are flooring and capping with correct percentiles & returning the correct output.'
assert round(np.quantile(sales['profit_outlier_capped'],0.03),4)==-144.632, 'Make sure that you are flooring and capping with correct percentiles & returning the correct output.'

In [57]:
# autograder cells , please do not alter/ delete /edit this cell

### Q.12) Business team is interested in knowing how many vendors have profit_outlier_capped >10000 & vendors with profit_outlier_capped <10000. Return the dataframe with 3 columns vendor  , profit_outlier_capped , profit_bins & sort it with 'profit_outlier_capped' in descending order.

Instructions:-
1. Bins should be in (a,b] format. [a,b) indicates that bin includes a but excudes b])
2. Make sure that for 1st interval it is **left inclusive** i.e [a,b].
3. Segment the ''profit_outlier_capped'' variable in 2 bins. (first bin with profit>10000 ,second bin with profit<10000).

In [58]:
sales.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,product_id,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,vendor_code,vendor,customer_name,city,state,country,segment,market,region,bin,shipping_mode_encoded,profit_outlier_capped
0,1,MX-2014-143658,2014-10-02,06/10/2014,5.00,SC-20575,OFF-LA-10002782,Low,Labels,"Hon File Folder Labels, Adjustable",13.08,3,0.00,4.56,1.03,Medium,VE_001,Accenco,Sonia Cooley,Mexico City,Distrito Federal,Mexico,Consumer,LATAM,LATAM-North,"[12.0, 37.0)",5.00,4.56
1,2,MX-2012-155047,2012-10-15,20/10/2012,5.00,KW-16570,FUR-FU-10004015,Medium,Furnishings,"Tenex Clock, Durable",252.16,8,0.00,90.72,13.45,Medium,VE_002,Ace Corporation,Kelly Williams,Dos Quebradas,Risaralda,Colombia,Consumer,LATAM,LATAM-South,"[186.0, 1200.0)",5.00,90.72
2,3,MX-2012-155047,2012-10-15,20/10/2012,5.00,KW-16570,FUR-BO-10002352,Low,Bookcases,"Ikea 3-Shelf Cabinet, Mobile",193.28,2,0.00,54.08,9.63,Medium,VE_003,Nero,Kelly Williams,Dos Quebradas,Risaralda,Colombia,Consumer,LATAM,LATAM-South,"[186.0, 1200.0)",5.00,54.08
3,4,MX-2012-155047,2012-10-15,20/10/2012,5.00,KW-16570,OFF-BI-10004428,Low,Binders,"Cardinal Binder, Clear",35.44,4,0.00,4.96,1.37,Medium,VE_004,Rushcorp,Kelly Williams,Dos Quebradas,Risaralda,Colombia,Consumer,LATAM,LATAM-South,"[12.0, 37.0)",5.00,4.96
4,5,MX-2012-155047,2012-10-15,20/10/2012,5.00,KW-16570,OFF-AR-10004594,Low,Art,"Sanford Canvas, Water Color",71.60,2,0.00,11.44,3.79,Medium,VE_005,Low Tide Corp,Kelly Williams,Dos Quebradas,Risaralda,Colombia,Consumer,LATAM,LATAM-South,"[37.0, 113.0)",5.00,11.44


In [59]:
sales.groupby("vendor").mean()

C:\Users\Dennis\AppData\Local\Temp\ipykernel_4220\1559516596.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sales.groupby("vendor").mean()


,row_id,ship_mode,sales,quantity,discount,profit,shipping_cost,shipping_mode_encoded,profit_outlier_capped
vendor,,,,,,,,,
Accenco,12058.25,5.00,128.34,3.72,0.11,16.56,12.46,5.00,15.97
Ace Corporation,5016.57,5.00,337.94,3.56,0.19,26.39,34.30,5.00,29.24
Alphacom,46172.99,5.00,109.34,2.38,0.16,12.83,9.59,5.00,12.07
Ansoft,15378.05,5.00,189.74,3.81,0.11,22.60,16.97,5.00,16.38
Apexi,41360.69,5.00,313.44,2.54,0.16,17.47,35.49,5.00,38.39
...,...,...,...,...,...,...,...,...,...
Vine Inc,19117.75,5.00,331.03,3.74,0.19,6.18,39.16,5.00,16.45
Vision Technologies,23934.32,5.00,426.65,3.73,0.13,77.87,44.11,5.00,48.30
Whizystems,14100.72,5.00,323.86,3.71,0.13,40.28,37.56,5.00,43.13


In [60]:
# Create two bins: one for profit > 10000 and one for profit <= 10000
bins = [0, 10000, float('inf')]
    
# Create labels for the bins
labels = ['<= 10000', '> 10000']

In [61]:
# Create a new column 'profit_bins' that segments 'profit_outlier_capped'
sales['profit_bins'] = pd.cut(sales['profit_outlier_capped'], bins=bins, labels=labels, right=False)

In [62]:
sales.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,product_id,category,sub_category,product_name,sales,quantity,discount,profit,shipping_cost,order_priority,vendor_code,vendor,customer_name,city,state,country,segment,market,region,bin,shipping_mode_encoded,profit_outlier_capped,profit_bins
0,1,MX-2014-143658,2014-10-02,06/10/2014,5.00,SC-20575,OFF-LA-10002782,Low,Labels,"Hon File Folder Labels, Adjustable",13.08,3,0.00,4.56,1.03,Medium,VE_001,Accenco,Sonia Cooley,Mexico City,Distrito Federal,Mexico,Consumer,LATAM,LATAM-North,"[12.0, 37.0)",5.00,4.56,<= 10000
1,2,MX-2012-155047,2012-10-15,20/10/2012,5.00,KW-16570,FUR-FU-10004015,Medium,Furnishings,"Tenex Clock, Durable",252.16,8,0.00,90.72,13.45,Medium,VE_002,Ace Corporation,Kelly Williams,Dos Quebradas,Risaralda,Colombia,Consumer,LATAM,LATAM-South,"[186.0, 1200.0)",5.00,90.72,<= 10000
2,3,MX-2012-155047,2012-10-15,20/10/2012,5.00,KW-16570,FUR-BO-10002352,Low,Bookcases,"Ikea 3-Shelf Cabinet, Mobile",193.28,2,0.00,54.08,9.63,Medium,VE_003,Nero,Kelly Williams,Dos Quebradas,Risaralda,Colombia,Consumer,LATAM,LATAM-South,"[186.0, 1200.0)",5.00,54.08,<= 10000
3,4,MX-2012-155047,2012-10-15,20/10/2012,5.00,KW-16570,OFF-BI-10004428,Low,Binders,"Cardinal Binder, Clear",35.44,4,0.00,4.96,1.37,Medium,VE_004,Rushcorp,Kelly Williams,Dos Quebradas,Risaralda,Colombia,Consumer,LATAM,LATAM-South,"[12.0, 37.0)",5.00,4.96,<= 10000
4,5,MX-2012-155047,2012-10-15,20/10/2012,5.00,KW-16570,OFF-AR-10004594,Low,Art,"Sanford Canvas, Water Color",71.60,2,0.00,11.44,3.79,Medium,VE_005,Low Tide Corp,Kelly Williams,Dos Quebradas,Risaralda,Colombia,Consumer,LATAM,LATAM-South,"[37.0, 113.0)",5.00,11.44,<= 10000


In [63]:
# Filter the dataframe to include only 'vendor', 'profit_outlier_capped', and 'profit_bins' columns
sales = sales[['vendor', 'profit_outlier_capped', 'profit_bins']]
sales

,vendor,profit_outlier_capped,profit_bins
0,Accenco,4.56,<= 10000
1,Ace Corporation,90.72,<= 10000
2,Nero,54.08,<= 10000
3,Rushcorp,4.96,<= 10000
4,Low Tide Corp,11.44,<= 10000
...,...,...,...
51285,Arcanestar,3.42,<= 10000
51286,Padlock Microsystems,18.42,<= 10000
51287,Stormking,20.88,<= 10000
51288,Arcanestar,33.84,<= 10000


In [64]:
sales['vendor'].value_counts()

Low Tide Corp       4284
Proton Solutions    3160
Venusystems         3071
Nimbleworks         2112
Accenco             2072
                    ... 
Oceanpaw             157
Luckydream           135
Moondustries         129
Apexman              122
Spiderwell           103
Name: vendor, Length: 65, dtype: int64

In [65]:
sales

,vendor,profit_outlier_capped,profit_bins
0,Accenco,4.56,<= 10000
1,Ace Corporation,90.72,<= 10000
2,Nero,54.08,<= 10000
3,Rushcorp,4.96,<= 10000
4,Low Tide Corp,11.44,<= 10000
...,...,...,...
51285,Arcanestar,3.42,<= 10000
51286,Padlock Microsystems,18.42,<= 10000
51287,Stormking,20.88,<= 10000
51288,Arcanestar,33.84,<= 10000


In [66]:
def profit_vendor(data,col):
    # Create two bins: one for profit > 10000 and one for profit <= 10000
    bins = [0, 10000, float('inf')]
    
    # Create labels for the bins
    labels = ['<= 10000', '> 10000']
    
    # Create a new column 'profit_bins' that segments 'profit_outlier_capped'
    data['profit_bins'] = pd.cut(data[col], bins=bins, labels=labels, right=False)
    
    # Filter the dataframe to include only 'vendor', 'profit_outlier_capped', and 'profit_bins' columns
    data = data[['vendor', 'profit_outlier_capped', 'profit_bins']]
    
    # Drop duplicate values in the 'vendor' column to make it distinct
    data['vendor'] = data['vendor'].drop_duplicates()
    
        
    # Reset the index to have a clean result
    data.reset_index(drop=True, inplace=True)
    
    # Sort the dataframe by 'profit_outlier_capped' in descending order
    data = data.sort_values(by='profit_outlier_capped', ascending=False)
    
    data = pd.DataFrame(data)
    
    return data

In [67]:
assert type(profit_vendor(data=sales,col='profit_outlier_capped'))==pd.DataFrame ,'Make sure that your output is a dataframe '
assert profit_vendor(data=sales,col='profit_outlier_capped').shape==(65,3) ,'Make sure that shape of your dataframe is correct'

C:\Users\Dennis\AppData\Local\Temp\ipykernel_4220\3095614636.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['profit_bins'] = pd.cut(data[col], bins=bins, labels=labels, right=False)
C:\Users\Dennis\AppData\Local\Temp\ipykernel_4220\3095614636.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['profit_bins'] = pd.cut(data[col], bins=bins, labels=labels, right=False)


AssertionError: Make sure that shape of your dataframe is correct

In [68]:
# autograder cells , please do not alter/ delete /edit this cell

#  55/100 points earned

# 